### Load Model

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "7"

import torch
print(torch.cuda.current_device())  # Should print 0 (because only device 7 is visible)
print(torch.cuda.device_count())  # Should print 1 (only one device is available)


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 16384 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "./llama_3.2/Llama_3.2_3B", # or choose "unsloth/Llama-3.2-1B-Instruct"
    model_name = "./data/SFT_ckpt/ckpt_outputs_response/10500/ckpt_epoch1_30/checkpoint-21091",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

### Add Lora Adapter

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)


### Data Prep


In [ ]:
use_rag = True  # Set to False if not using RAG


ABC_prompt_rag = """You are a CAD model generation assistant trained to produce STEP (.step) files based on textual descriptions. Given the following object description and relevant retrieved CAD data, generate a STEP file that accurately represents the described object.


### caption:
{}

### retrieved relevant step file:
{}

### output:
{}"""

ABC_prompt_no_rag = """You are a CAD model generation assistant trained to produce STEP (.step) files based on textual descriptions. Given the following object description, generate a STEP file that accurately represents the described object.

### caption:
{}

### output:
{}"""

ABC_prompt_no_output = """You are a CAD model generation assistant trained to produce STEP (.step) files based on textual descriptions. Given the following object description, generate a STEP file that accurately represents the described object.

### caption:
{}

### output:
"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["caption"]
    # inputs       = examples["relavant_step_file"]
    outputs      = examples["output"]
    # instructions = examples["instruction"]
    # inputs       = examples["input"]
    # outputs      = examples["output"]
    texts = []
    if use_rag:
        inputs = examples["relavant_step_file"]
        for instruction, input, output in zip(instructions, inputs, outputs):
            text = ABC_prompt_rag.format(instruction, input, output) + EOS_TOKEN
            texts.append(text)
    else:
        for instruction, output in zip(instructions, outputs):
            text = ABC_prompt_no_rag.format(instruction, output) + EOS_TOKEN
            texts.append(text)


    return { "text" : texts, }
pass

from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")

dataset = load_dataset(path="./data/abc_rag/20500", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True,)
test_dataset = load_dataset(path="./data/abc_rag/20500", split="test")
test_dataset = test_dataset.map(formatting_prompts_func, batched=True)


In [ ]:
# Print the entire dataset
print(dataset)

# Print the first data entry
print(dataset[0]['text'])

### Train the Model

In [ ]:
import wandb
wandb.login()  # paste your API key when prompted
wandb.init(
  project="cad_llm",    # this will be created under YOUR username by default
  # no need to pass `entity` if you want it under your personal account
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# trainer = SFTTrainer(
#     model = model,
#     tokenizer = tokenizer,
#     train_dataset = dataset,
#     dataset_text_field = "text",
#     max_seq_length = max_seq_length,
#     data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
#     dataset_num_proc = 2,
#     packing = False, # Can make training 5x faster for short sequences.
#     args = TrainingArguments(
#         per_device_train_batch_size = 2,
#         gradient_accumulation_steps = 4,
#         # warmup_steps = 110,
#         # num_train_epochs = 5, # Set this for 1 full training run.
#         max_steps = 10,
#         learning_rate = 2e-4,
#         fp16 = not is_bfloat16_supported(),
#         bf16 = is_bfloat16_supported(),
#         logging_steps = 5,
#         optim = "adamw_8bit",
#         weight_decay = 0.01,
#         lr_scheduler_type = "linear",
#         seed = 3407,
#         output_dir = "ckpt_outputs_response/ckpt_epoch11",
#         report_to = "none", # Use this for WandB etc
#     ),
# )

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = test_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 2, #4,
        warmup_steps = 1,#5,
        # warmup_steps = 3,
        # num_train_epochs = 1, # Set this for 1 full tranining run.
        # num_train_epochs = 2,
        max_steps = 10,#100,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "./data/SFT_ckpt/tmp_test_purpose",
        report_to = "none", # Use this for WandB etc
        # 关键改动:
        # report_to="wandb",              # 打开 wandb 报表
        # run_name="cad_sft_experiment",   # wandb 中的 run 名称
        save_strategy="epoch",          # 每个 epoch 保存一次模型
        # evaluation_strategy="steps",  # Run test loss calculation during training
        # eval_steps=12,  # Calculate test loss every 5 steps
        # remove_unused_columns=False,
    ),
)

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "### caption:\n",
    response_part = "### output:\n",
)

In [ ]:
print(trainer.train_dataset)
print(trainer.eval_dataset)

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

In [ ]:
trainer_stats = trainer.train()


In [ ]:
import pandas as pd

stats_dict = {
    "epoch": [],
    "loss": [],
    "eval_loss": [],
    "learning_rate": [],
    "step": []
}

if hasattr(trainer, "state") and hasattr(trainer.state, "log_history"):
    last_eval_loss = None  # To keep track of the latest test loss
    for log in trainer.state.log_history:
        if "loss" in log:
            stats_dict["epoch"].append(log.get("epoch", "N/A"))
            stats_dict["loss"].append(log.get("loss", "N/A"))
            stats_dict["learning_rate"].append(log.get("learning_rate", "N/A"))
            stats_dict["step"].append(log.get("step", "N/A"))

            # Append last test loss available at this step
            stats_dict["eval_loss"].append(last_eval_loss)
        
        if "eval_loss" in log:
            last_eval_loss = log["eval_loss"]  # Update test loss when found

stats_df = pd.DataFrame(stats_dict)

print(stats_df)

### Inference


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 16384 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./llama_3.2/Llama_3.2_3B", # or choose "unsloth/Llama-3.2-1B-Instruct"
    # model_name = "./data/SFT_ckpt/ckpt_outputs_response/10500/ckpt_epoch1_30/checkpoint-21091",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
# alpaca_prompt = Copied from above
ABC_prompt = """You are a CAD model generation assistant trained to produce STEP (.step) files based on textual descriptions. Given the following object description and relevant retrieved CAD data, generate a STEP file that accurately represents the described object.


### caption:
{}

### retrieved relavant step file:
{}

### output:
{}"""

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    ABC_prompt.format(
        "A flat, elongated bar with multiple circular holes along its length", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 200, use_cache = True)
tokenizer.batch_decode(outputs)

### Saving, loading finetuned models

In [ ]:
model.save_pretrained("lora_model_response") # Local saving
tokenizer.save_pretrained("lora_model_response")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
# Inference (RAG): ./data/SFT_ckpt/ckpt_outputs_response_rag/10500/ckpt_epoch1_30/checkpoint-25676
# restart all the kernels before running
from unsloth import FastLanguageModel
import torch
max_seq_length = 16384 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        #model_name = "lora_model_rag", # YOUR MODEL YOU USED FOR TRAINING
        model_name = "./data/merged_model/dfs-7200",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# ABC_prompt = """You are a CAD model generation assistant trained to produce STEP (.step) files based on textual descriptions. Given the following object description and relevant retrieved CAD data, generate a STEP file that accurately represents the described object.


# ### caption:
# {}

# ### retrieved relavant step file:
# {}

# ### output:
# {}"""

ABC_prompt = """You are a CAD model generation assistant trained to produce STEP (.step) files based on textual descriptions. Given the following object description and relevant retrieved CAD data, generate a STEP file that accurately represents the described object.


### caption:
{}

### retrieved relavant step file:
{}

### output:
{}"""


inputs = tokenizer(
[
    ABC_prompt.format(
        "A rectangular block with a slanted top surface.", # instruction
        "DATA;\n#1 = MECHANICAL_DESIGN_GEOMETRIC_PRESENTATION_REPRESENTATION( ' ', ( #10, #11, #12, #13, #14, #15 ), #6 );\n#2 = PRODUCT_DEFINITION_CONTEXT( '', #16, 'design' );\n#3 = APPLICATION_PROTOCOL_DEFINITION( 'international standard', 'automotive_design', 2001, #16 );\n#4 = PRODUCT_CATEGORY_RELATIONSHIP( 'NONE', 'NONE', #17, #18 );\n#5 = SHAPE_DEFINITION_REPRESENTATION( #19, #20 );\n#6 =  ( GEOMETRIC_REPRESENTATION_CONTEXT( 3 )GLOBAL_UNCERTAINTY_ASSIGNED_CONTEXT( ( #23 ) )GLOBAL_UNIT_ASSIGNED_CONTEXT( ( #25, #26, #27 ) )REPRESENTATION_CONTEXT( 'NONE', 'WORKSPACE' ) );\n#10 = STYLED_ITEM( '', ( #29 ), #30 );\n#11 = STYLED_ITEM( '', ( #31 ), #32 );\n#12 = STYLED_ITEM( '', ( #33 ), #34 );\n#13 = STYLED_ITEM( '', ( #35 ), #36 );\n#14 = STYLED_ITEM( '', ( #37 ), #38 );\n#15 = STYLED_ITEM( '', ( #39 ), #40 );\n#16 = APPLICATION_CONTEXT( 'core data for automotive mechanical design processes' );\n#17 = PRODUCT_CATEGORY( 'part', 'NONE' );\n#18 = PRODUCT_RELATED_PRODUCT_CATEGORY( 'detail', ' ', ( #41 ) );\n#19 = PRODUCT_DEFINITION_SHAPE( 'NONE', 'NONE', #42 );\n#20 = ADVANCED_BREP_SHAPE_REPRESENTATION( 'Reference Geometry', ( #43, #44 ), #6 );\n#23 = UNCERTAINTY_MEASURE_WITH_UNIT( LENGTH_MEASURE( 1.00000000000000E-17 ), #25, '', '' );\n#25 =  ( CONVERSION_BASED_UNIT( 'METRE', #47 )LENGTH_UNIT(  )NAMED_UNIT( #50 ) );\n#26 =  ( NAMED_UNIT( #52 )PLANE_ANGLE_UNIT(  )SI_UNIT( $, .RADIAN. ) );\n#27 =  ( NAMED_UNIT( #52 )SI_UNIT( $, .STERADIAN. )SOLID_ANGLE_UNIT(  ) );\n#29 = PRESENTATION_STYLE_ASSIGNMENT( ( #58 ) );\n#30 = ADVANCED_FACE( '', ( #59 ), #60, .T. );\n#31 = PRESENTATION_STYLE_ASSIGNMENT( ( #61 ) );\n#32 = ADVANCED_FACE( '', ( #62 ), #63, .F. );\n#33 = PRESENTATION_STYLE_ASSIGNMENT( ( #64 ) );\n#34 = ADVANCED_FACE( '', ( #65 ), #66, .F. );\n#35 = PRESENTATION_STYLE_ASSIGNMENT( ( #67 ) );\n#36 = ADVANCED_FACE( '', ( #68 ), #69, .T. );\n#37 = PRESENTATION_STYLE_ASSIGNMENT( ( #70 ) );\n#38 = ADVANCED_FACE( '', ( #71 ), #72, .T. );\n#39 = PRESENTATION_STYLE_ASSIGNMENT( ( #73 ) );\n#40 = ADVANCED_FACE( '', ( #74 ), #75, .F. );\n#41 = PRODUCT( 'Reference Geometry', 'Reference Geometry', 'PART-Reference Geometry-DESC', ( #76 ) );\n#42 = PRODUCT_DEFINITION( 'NONE', 'NONE', #77, #2 );\n#43 = MANIFOLD_SOLID_BREP( 'Reference Geometry', #78 );\n#44 = AXIS2_PLACEMENT_3D( '', #79, #80, #81 );\n#47 = LENGTH_MEASURE_WITH_UNIT( LENGTH_MEASURE( 1.00000000000000 ), #82 );\n#50 = DIMENSIONAL_EXPONENTS( 1.00000000000000, 0.00000000000000, 0.00000000000000, 0.00000000000000, 0.00000000000000, 0.00000000000000, 0.00000000000000 );\n#52 = DIMENSIONAL_EXPONENTS( 0.00000000000000, 0.00000000000000, 0.00000000000000, 0.00000000000000, 0.00000000000000, 0.00000000000000, 0.00000000000000 );\n#58 = SURFACE_STYLE_USAGE( .BOTH., #83 );\n#59 = FACE_OUTER_BOUND( '', #84, .T. );\n#60 = PLANE( '', #85 );\n#61 = SURFACE_STYLE_USAGE( .BOTH., #86 );\n#62 = FACE_OUTER_BOUND( '', #87, .T. );\n#63 = PLANE( '', #88 );\n#64 = SURFACE_STYLE_USAGE( .BOTH., #89 );\n#65 = FACE_OUTER_BOUND( '', #90, .T. );\n#66 = PLANE( '', #91 );\n#67 = SURFACE_STYLE_USAGE( .BOTH., #92 );\n#68 = FACE_OUTER_BOUND( '', #93, .T. );\n#69 = PLANE( '', #94 );\n#70 = SURFACE_STYLE_USAGE( .BOTH., #95 );\n#71 = FACE_OUTER_BOUND( '', #96, .T. );\n#72 = PLANE( '', #97 );\n#73 = SURFACE_STYLE_USAGE( .BOTH., #98 );\n#74 = FACE_OUTER_BOUND( '', #99, .T. );\n#75 = PLANE( '', #100 );\n#76 = PRODUCT_CONTEXT( '', #16, 'mechanical' );\n#77 = PRODUCT_DEFINITION_FORMATION_WITH_SPECIFIED_SOURCE( ' ', 'NONE', #41, .NOT_KNOWN. );\n#78 = CLOSED_SHELL( '', ( #38, #36, #34, #32, #40, #30 ) );\n#79 = CARTESIAN_POINT( '', ( 0.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#80 = DIRECTION( '', ( 0.00000000000000, 0.00000000000000, 1.00000000000000 ) );\n#81 = DIRECTION( '', ( 1.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#82 =  ( LENGTH_UNIT(  )NAMED_UNIT( #50 )SI_UNIT( $, .METRE. ) );\n#83 = SURFACE_SIDE_STYLE( '', ( #102 ) );\n#84 = EDGE_LOOP( '', ( #103, #104, #105, #106 ) );\n#85 = AXIS2_PLACEMENT_3D( '', #107, #108, #109 );\n#86 = SURFACE_SIDE_STYLE( '', ( #110 ) );\n#87 = EDGE_LOOP( '', ( #111, #112, #113, #114 ) );\n#88 = AXIS2_PLACEMENT_3D( '', #115, #116, #117 );\n#89 = SURFACE_SIDE_STYLE( '', ( #118 ) );\n#90 = EDGE_LOOP( '', ( #119, #120, #121, #122 ) );\n#91 = AXIS2_PLACEMENT_3D( '', #123, #124, #125 );\n#92 = SURFACE_SIDE_STYLE( '', ( #126 ) );\n#93 = EDGE_LOOP( '', ( #127, #128, #129, #130 ) );\n#94 = AXIS2_PLACEMENT_3D( '', #131, #132, #133 );\n#95 = SURFACE_SIDE_STYLE( '', ( #134 ) );\n#96 = EDGE_LOOP( '', ( #135, #136, #137, #138 ) );\n#97 = AXIS2_PLACEMENT_3D( '', #139, #140, #141 );\n#98 = SURFACE_SIDE_STYLE( '', ( #142 ) );\n#99 = EDGE_LOOP( '', ( #143, #144, #145, #146 ) );\n#100 = AXIS2_PLACEMENT_3D( '', #147, #148, #149 );\n#102 = SURFACE_STYLE_FILL_AREA( #150 );\n#103 = ORIENTED_EDGE( '', *, *, #151, .F. );\n#104 = ORIENTED_EDGE( '', *, *, #152, .F. );\n#105 = ORIENTED_EDGE( '', *, *, #153, .T. );\n#106 = ORIENTED_EDGE( '', *, *, #154, .T. );\n#107 = CARTESIAN_POINT( '', ( 0.00500000000000000, -0.0250000000000000, 0.00500000000000000 ) );\n#108 = DIRECTION( '', ( 0.00000000000000, -1.00000000000000, 0.00000000000000 ) );\n#109 = DIRECTION( '', ( 1.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#110 = SURFACE_STYLE_FILL_AREA( #155 );\n#111 = ORIENTED_EDGE( '', *, *, #156, .T. );\n#112 = ORIENTED_EDGE( '', *, *, #157, .F. );\n#113 = ORIENTED_EDGE( '', *, *, #154, .F. );\n#114 = ORIENTED_EDGE( '', *, *, #158, .T. );\n#115 = CARTESIAN_POINT( '', ( 0.0100000000000000, -0.0250000000000000, 0.00646000046755786 ) );\n#116 = DIRECTION( '', ( -1.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#117 = DIRECTION( '', ( 0.00000000000000, 0.00000000000000, 1.00000000000000 ) );\n#118 = SURFACE_STYLE_FILL_AREA( #159 );\n#119 = ORIENTED_EDGE( '', *, *, #160, .T. );\n#120 = ORIENTED_EDGE( '', *, *, #158, .F. );\n#121 = ORIENTED_EDGE( '', *, *, #153, .F. );\n#122 = ORIENTED_EDGE( '', *, *, #161, .T. );\n#123 = CARTESIAN_POINT( '', ( 0.00500000000000000, -0.0250000000000000, 0.00000000000000 ) );\n#124 = DIRECTION( '', ( 0.00000000000000, 0.00000000000000, 1.00000000000000 ) );\n#125 = DIRECTION( '', ( 1.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#126 = SURFACE_STYLE_FILL_AREA( #162 );\n#127 = ORIENTED_EDGE( '', *, *, #163, .F. );\n#128 = ORIENTED_EDGE( '', *, *, #161, .F. );\n#129 = ORIENTED_EDGE( '', *, *, #152, .T. );\n#130 = ORIENTED_EDGE( '', *, *, #164, .T. );\n#131 = CARTESIAN_POINT( '', ( 0.00000000000000, -0.0250000000000000, 0.00646000046755786 ) );\n#132 = DIRECTION( '', ( -1.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#133 = DIRECTION( '', ( 0.00000000000000, 0.00000000000000, 1.00000000000000 ) );\n#134 = SURFACE_STYLE_FILL_AREA( #165 );\n#135 = ORIENTED_EDGE( '', *, *, #166, .F. );\n#136 = ORIENTED_EDGE( '', *, *, #164, .F. );\n#137 = ORIENTED_EDGE( '', *, *, #151, .T. );\n#138 = ORIENTED_EDGE( '', *, *, #157, .T. );\n#139 = CARTESIAN_POINT( '', ( 0.00500000000000000, -0.0250000000000000, 0.0100000000000000 ) );\n#140 = DIRECTION( '', ( 0.00000000000000, 0.00000000000000, 1.00000000000000 ) );\n#141 = DIRECTION( '', ( 1.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#142 = SURFACE_STYLE_FILL_AREA( #167 );\n#143 = ORIENTED_EDGE( '', *, *, #166, .T. );\n#144 = ORIENTED_EDGE( '', *, *, #156, .F. );\n#145 = ORIENTED_EDGE( '', *, *, #160, .F. );\n#146 = ORIENTED_EDGE( '', *, *, #163, .T. );\n#147 = CARTESIAN_POINT( '', ( 0.00500000000000000, 0.00000000000000, 0.00500000000000000 ) );\n#148 = DIRECTION( '', ( 0.00000000000000, -1.00000000000000, 0.00000000000000 ) );\n#149 = DIRECTION( '', ( 1.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#150 = FILL_AREA_STYLE( '', ( #168 ) );\n#151 = EDGE_CURVE( '', #169, #170, #171, .T. );\n#152 = EDGE_CURVE( '', #172, #169, #173, .T. );\n#153 = EDGE_CURVE( '', #172, #174, #175, .T. );\n#154 = EDGE_CURVE( '', #174, #170, #176, .T. );\n#155 = FILL_AREA_STYLE( '', ( #177 ) );\n#156 = EDGE_CURVE( '', #178, #179, #180, .T. );\n#157 = EDGE_CURVE( '', #170, #179, #181, .T. );\n#158 = EDGE_CURVE( '', #174, #178, #182, .T. );\n#159 = FILL_AREA_STYLE( '', ( #183 ) );\n#160 = EDGE_CURVE( '', #184, #178, #185, .T. );\n#161 = EDGE_CURVE( '', #172, #184, #186, .T. );\n#162 = FILL_AREA_STYLE( '', ( #187 ) );\n#163 = EDGE_CURVE( '', #184, #188, #189, .T. );\n#164 = EDGE_CURVE( '', #169, #188, #190, .T. );\n#165 = FILL_AREA_STYLE( '', ( #191 ) );\n#166 = EDGE_CURVE( '', #188, #179, #192, .T. );\n#167 = FILL_AREA_STYLE( '', ( #193 ) );\n#168 = FILL_AREA_STYLE_COLOUR( '', #194 );\n#169 = VERTEX_POINT( '', #195 );\n#170 = VERTEX_POINT( '', #196 );\n#171 = LINE( '', #197, #198 );\n#172 = VERTEX_POINT( '', #199 );\n#173 = LINE( '', #200, #201 );\n#174 = VERTEX_POINT( '', #202 );\n#175 = LINE( '', #203, #204 );\n#176 = LINE( '', #205, #206 );\n#177 = FILL_AREA_STYLE_COLOUR( '', #207 );\n#178 = VERTEX_POINT( '', #208 );\n#179 = VERTEX_POINT( '', #209 );\n#180 = LINE( '', #210, #211 );\n#181 = LINE( '', #212, #213 );\n#182 = LINE( '', #214, #215 );\n#183 = FILL_AREA_STYLE_COLOUR( '', #216 );\n#184 = VERTEX_POINT( '', #217 );\n#185 = LINE( '', #218, #219 );\n#186 = LINE( '', #220, #221 );\n#187 = FILL_AREA_STYLE_COLOUR( '', #222 );\n#188 = VERTEX_POINT( '', #223 );\n#189 = LINE( '', #224, #225 );\n#190 = LINE( '', #226, #227 );\n#191 = FILL_AREA_STYLE_COLOUR( '', #228 );\n#192 = LINE( '', #229, #230 );\n#193 = FILL_AREA_STYLE_COLOUR( '', #231 );\n#194 = COLOUR_RGB( '', 0.603921592200000, 0.647058844600000, 0.686274528500000 );\n#195 = CARTESIAN_POINT( '', ( 0.00000000000000, -0.0250000000000000, 0.0100000000000000 ) );\n#196 = CARTESIAN_POINT( '', ( 0.0100000000000000, -0.0250000000000000, 0.0100000000000000 ) );\n#197 = CARTESIAN_POINT( '', ( 0.00500000000000000, -0.0250000000000000, 0.0100000000000000 ) );\n#198 = VECTOR( '', #232, 1.00000000000000 );\n#199 = CARTESIAN_POINT( '', ( 0.00000000000000, -0.0250000000000000, 0.00000000000000 ) );\n#200 = CARTESIAN_POINT( '', ( 0.00000000000000, -0.0250000000000000, 0.00646000046755786 ) );\n#201 = VECTOR( '', #233, 1.00000000000000 );\n#202 = CARTESIAN_POINT( '', ( 0.0100000000000000, -0.0250000000000000, 0.00000000000000 ) );\n#203 = CARTESIAN_POINT( '', ( 0.00500000000000000, -0.0250000000000000, 0.00000000000000 ) );\n#204 = VECTOR( '', #234, 1.00000000000000 );\n#205 = CARTESIAN_POINT( '', ( 0.0100000000000000, -0.0250000000000000, 0.00646000046755786 ) );\n#206 = VECTOR( '', #235, 1.00000000000000 );\n#207 = COLOUR_RGB( '', 0.603921592200000, 0.647058844600000, 0.686274528500000 );\n#208 = CARTESIAN_POINT( '', ( 0.0100000000000000, 0.00000000000000, 0.00000000000000 ) );\n#209 = CARTESIAN_POINT( '', ( 0.0100000000000000, 0.00000000000000, 0.0100000000000000 ) );\n#210 = CARTESIAN_POINT( '', ( 0.0100000000000000, 0.00000000000000, 0.00646000046755786 ) );\n#211 = VECTOR( '', #236, 1.00000000000000 );\n#212 = CARTESIAN_POINT( '', ( 0.0100000000000000, -0.0250000000000000, 0.0100000000000000 ) );\n#213 = VECTOR( '', #237, 1.00000000000000 );\n#214 = CARTESIAN_POINT( '', ( 0.0100000000000000, -0.0250000000000000, 0.00000000000000 ) );\n#215 = VECTOR( '', #238, 1.00000000000000 );\n#216 = COLOUR_RGB( '', 0.603921592200000, 0.647058844600000, 0.686274528500000 );\n#217 = CARTESIAN_POINT( '', ( 0.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#218 = CARTESIAN_POINT( '', ( 0.00500000000000000, 0.00000000000000, 0.00000000000000 ) );\n#219 = VECTOR( '', #239, 1.00000000000000 );\n#220 = CARTESIAN_POINT( '', ( 0.00000000000000, -0.0250000000000000, 0.00000000000000 ) );\n#221 = VECTOR( '', #240, 1.00000000000000 );\n#222 = COLOUR_RGB( '', 0.603921592200000, 0.647058844600000, 0.686274528500000 );\n#223 = CARTESIAN_POINT( '', ( 0.00000000000000, 0.00000000000000, 0.0100000000000000 ) );\n#224 = CARTESIAN_POINT( '', ( 0.00000000000000, 0.00000000000000, 0.00646000046755786 ) );\n#225 = VECTOR( '', #241, 1.00000000000000 );\n#226 = CARTESIAN_POINT( '', ( 0.00000000000000, -0.0250000000000000, 0.0100000000000000 ) );\n#227 = VECTOR( '', #242, 1.00000000000000 );\n#228 = COLOUR_RGB( '', 0.603921592200000, 0.647058844600000, 0.686274528500000 );\n#229 = CARTESIAN_POINT( '', ( 0.00500000000000000, 0.00000000000000, 0.0100000000000000 ) );\n#230 = VECTOR( '', #243, 1.00000000000000 );\n#231 = COLOUR_RGB( '', 0.603921592200000, 0.647058844600000, 0.686274528500000 );\n#232 = DIRECTION( '', ( 1.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#233 = DIRECTION( '', ( 0.00000000000000, 0.00000000000000, 1.00000000000000 ) );\n#234 = DIRECTION( '', ( 1.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#235 = DIRECTION( '', ( 0.00000000000000, 0.00000000000000, 1.00000000000000 ) );\n#236 = DIRECTION( '', ( 0.00000000000000, 0.00000000000000, 1.00000000000000 ) );\n#237 = DIRECTION( '', ( 0.00000000000000, 1.00000000000000, 0.00000000000000 ) );\n#238 = DIRECTION( '', ( 0.00000000000000, 1.00000000000000, 0.00000000000000 ) );\n#239 = DIRECTION( '', ( 1.00000000000000, 0.00000000000000, 0.00000000000000 ) );\n#240 = DIRECTION( '', ( 0.00000000000000, 1.00000000000000, 0.00000000000000 ) );\n#241 = DIRECTION( '', ( 0.00000000000000, 0.00000000000000, 1.00000000000000 ) );\n#242 = DIRECTION( '', ( 0.00000000000000, 1.00000000000000, 0.00000000000000 ) );\n#243 = DIRECTION( '', ( 1.00000000000000, 0.00000000000000, 0.00000000000000 ) );\nENDSEC;\nEND-ISO-10303-21;",
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2000)

### Plot loss

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
train_csv_file = "./training_log/training_stats_3500_epoch10.csv"
df_train = pd.read_csv(train_csv_file)

eval_csv_file = "./training_log/eval_stats_3500_epoch10.csv"
df_eval = pd.read_csv(eval_csv_file)

# Compute average loss per epoch
epochs_train = df_train["epoch"]
losses_train = df_train["loss"]

# Create a dictionary to store sum and count of losses for each epoch
epoch_loss_train = {}
for epoch, loss in zip(epochs_train, losses_train):
    rounded_epoch = int(epoch)  # Convert fractional epoch to integer epoch number
    if rounded_epoch not in epoch_loss_train:
        epoch_loss_train[rounded_epoch] = {"sum": 0, "count": 0}
    epoch_loss_train[rounded_epoch]["sum"] += loss
    epoch_loss_train[rounded_epoch]["count"] += 1

# Compute average loss for each epoch
epoch_avg_loss_train = {epoch: epoch_loss_train[epoch]["sum"] / epoch_loss_train[epoch]["count"] for epoch in epoch_loss_train}

# Compute average evaluation loss per epoch
epochs_eval = df_eval["epoch"]
losses_eval = df_eval["eval_loss"]

# Dictionary to store sum and count of evaluation losses per epoch
epoch_loss_eval = {}
for epoch, loss in zip(epochs_eval, losses_eval):
    rounded_epoch = int(epoch)  # Convert fractional epoch to integer epoch number
    if rounded_epoch not in epoch_loss_eval:
        epoch_loss_eval[rounded_epoch] = {"sum": 0, "count": 0}
    epoch_loss_eval[rounded_epoch]["sum"] += loss
    epoch_loss_eval[rounded_epoch]["count"] += 1

# Compute average evaluation loss per epoch
epoch_avg_loss_eval = {epoch: epoch_loss_eval[epoch]["sum"] / epoch_loss_eval[epoch]["count"] for epoch in epoch_loss_eval}

# Plot training and evaluation loss curves
plt.figure(figsize=(8, 5))
plt.plot(epoch_avg_loss_train.keys(), epoch_avg_loss_train.values(), marker="o", linestyle="-", label="Training Loss", color="blue")
plt.plot(epoch_avg_loss_eval.keys(), epoch_avg_loss_eval.values(), marker="s", linestyle="--", label="Evaluation Loss", color="red")

# Labels and title
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Evaluation Loss per Epoch")
plt.legend()
plt.grid()

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# File paths
train_csv_1_10 = "./training_log/training_stats_3500_epoch10.csv"
train_csv_11_20 = "./training_log/training_stats_3500_epoch11_20.csv"
train_csv_21_25 = "./training_log/training_stats_3500_epoch21_25.csv"
train_csv_26_30 = "./training_log/training_stats_3500_epoch26_30.csv"
eval_csv = "./training_log/eval_loss_ckpt.csv"

# Load training loss for epochs 1-10
df_train_1_10 = pd.read_csv(train_csv_1_10)
epochs_train_1_10 = df_train_1_10["epoch"]
losses_train_1_10 = df_train_1_10["loss"]

# Compute average training loss per epoch (1-10)
epoch_loss_train_1_10 = {}
for epoch, loss in zip(epochs_train_1_10, losses_train_1_10):
    rounded_epoch = int(epoch)+1  
    if rounded_epoch not in epoch_loss_train_1_10:
        epoch_loss_train_1_10[rounded_epoch] = {"sum": 0, "count": 0}
    epoch_loss_train_1_10[rounded_epoch]["sum"] += loss
    epoch_loss_train_1_10[rounded_epoch]["count"] += 1

epoch_avg_loss_train_1_10 = {
    epoch: epoch_loss_train_1_10[epoch]["sum"] / epoch_loss_train_1_10[epoch]["count"]
    for epoch in epoch_loss_train_1_10
}
print(epoch_avg_loss_train_1_10)

# Load training loss for epochs 11-20
df_train_11_20 = pd.read_csv(train_csv_11_20)
epochs_train_11_20 = df_train_11_20["epoch"]
losses_train_11_20 = df_train_11_20["loss"]

# Compute average training loss per epoch (11-20)
epoch_loss_train_11_20 = {}
for epoch, loss in zip(epochs_train_11_20, losses_train_11_20):
    rounded_epoch = int(epoch) + 11  
    if rounded_epoch not in epoch_loss_train_11_20:
        epoch_loss_train_11_20[rounded_epoch] = {"sum": 0, "count": 0}
    epoch_loss_train_11_20[rounded_epoch]["sum"] += loss
    epoch_loss_train_11_20[rounded_epoch]["count"] += 1

epoch_avg_loss_train_11_20 = {
    epoch: epoch_loss_train_11_20[epoch]["sum"] / epoch_loss_train_11_20[epoch]["count"]
    for epoch in epoch_loss_train_11_20
}

# Load training loss for epochs 21-25
df_train_21_25 = pd.read_csv(train_csv_21_25)
epochs_train_21_25 = df_train_21_25["epoch"]
losses_train_21_25 = df_train_21_25["loss"]

# Compute average training loss per epoch (11-20)
epoch_loss_train_21_25 = {}
for epoch, loss in zip(epochs_train_21_25, losses_train_21_25):
    rounded_epoch = int(epoch) + 21  
    if rounded_epoch not in epoch_loss_train_21_25:
        epoch_loss_train_21_25[rounded_epoch] = {"sum": 0, "count": 0}
    epoch_loss_train_21_25[rounded_epoch]["sum"] += loss
    epoch_loss_train_21_25[rounded_epoch]["count"] += 1

epoch_avg_loss_train_21_25 = {
    epoch: epoch_loss_train_21_25[epoch]["sum"] / epoch_loss_train_21_25[epoch]["count"]
    for epoch in epoch_loss_train_21_25
}

# Load training loss for epochs 26-30
df_train_26_30 = pd.read_csv(train_csv_26_30)
epochs_train_26_30 = df_train_26_30["epoch"]
losses_train_26_30 = df_train_26_30["loss"]

# Compute average training loss per epoch (11-20)
epoch_loss_train_26_30 = {}
for epoch, loss in zip(epochs_train_26_30, losses_train_26_30):
    rounded_epoch = int(epoch) + 26  
    if rounded_epoch not in epoch_loss_train_26_30:
        epoch_loss_train_26_30[rounded_epoch] = {"sum": 0, "count": 0}
    epoch_loss_train_26_30[rounded_epoch]["sum"] += loss
    epoch_loss_train_26_30[rounded_epoch]["count"] += 1

epoch_avg_loss_train_26_30 = {
    epoch: epoch_loss_train_26_30[epoch]["sum"] / epoch_loss_train_26_30[epoch]["count"]
    for epoch in epoch_loss_train_26_30
}
# print(epoch_avg_loss_train_11_20)
# Merge training losses from epochs 1-20
epoch_avg_loss_train = {**epoch_avg_loss_train_1_10, **epoch_avg_loss_train_11_20, **epoch_avg_loss_train_21_25, **epoch_avg_loss_train_26_30}
print(epoch_avg_loss_train)

# Load evaluation loss for epochs 1-20
df_eval = pd.read_csv(eval_csv)
epochs_eval = df_eval["epoch"]
losses_eval = df_eval["eval_loss"]

# Compute average evaluation loss per epoch (1-20)
epoch_loss_eval = {}
for epoch, loss in zip(epochs_eval, losses_eval):
    rounded_epoch = int(epoch)  
    if rounded_epoch not in epoch_loss_eval:
        epoch_loss_eval[rounded_epoch] = {"sum": 0, "count": 0}
    epoch_loss_eval[rounded_epoch]["sum"] += loss
    epoch_loss_eval[rounded_epoch]["count"] += 1

epoch_avg_loss_eval = {
    epoch: epoch_loss_eval[epoch]["sum"] / epoch_loss_eval[epoch]["count"]
    for epoch in epoch_loss_eval
}

# Plot training and evaluation loss curves
plt.figure(figsize=(8, 5))
plt.plot(epoch_avg_loss_train.keys(), epoch_avg_loss_train.values(), marker="o", linestyle="-", label="Training Loss", color="blue")
plt.plot(epoch_avg_loss_eval.keys(), epoch_avg_loss_eval.values(), marker="s", linestyle="--", label="Evaluation Loss", color="red")

# Labels and title
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Evaluation Loss per Epoch (1-30)")
plt.legend()
plt.grid()

# Show the plot
plt.show()


In the above figure, epoch1-10 and epoch11-20 are trained separately, with lr strategy being set "linear" and "constant" namely. (epoch1-10: 2e-4 -> 0, epoch11-20: 2e-05)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# File paths
train_csv = "./training_log/training_stats_7000_epoch40_rag.csv"
eval_csv = "./training_log/eval_loss_7000_epoch_rag.csv"

# Load training loss for epochs 1-10
df_train = pd.read_csv(train_csv)
epochs_train = df_train["epoch"]
losses_train = df_train["loss"]

# Compute average training loss per epoch (1-10)
epoch_loss_train = {}
for epoch, loss in zip(epochs_train, losses_train):
    rounded_epoch = int(epoch)+1  
    if rounded_epoch not in epoch_loss_train:
        epoch_loss_train[rounded_epoch] = {"sum": 0, "count": 0}
    epoch_loss_train[rounded_epoch]["sum"] += loss
    epoch_loss_train[rounded_epoch]["count"] += 1

epoch_avg_loss_train = {
    epoch: epoch_loss_train[epoch]["sum"] / epoch_loss_train[epoch]["count"]
    for epoch in epoch_loss_train
}
print(epoch_avg_loss_train)




# Load evaluation loss for epochs 1-20
df_eval = pd.read_csv(eval_csv)
epochs_eval = df_eval["epoch"]
losses_eval = df_eval["eval_loss"]

# Compute average evaluation loss per epoch (1-20)
epoch_loss_eval = {}
for epoch, loss in zip(epochs_eval, losses_eval):
    rounded_epoch = int(epoch)  
    if rounded_epoch not in epoch_loss_eval:
        epoch_loss_eval[rounded_epoch] = {"sum": 0, "count": 0}
    epoch_loss_eval[rounded_epoch]["sum"] += loss
    epoch_loss_eval[rounded_epoch]["count"] += 1

epoch_avg_loss_eval = {
    epoch: epoch_loss_eval[epoch]["sum"] / epoch_loss_eval[epoch]["count"]
    for epoch in epoch_loss_eval
}

# Plot training and evaluation loss curves
plt.figure(figsize=(8, 5))
plt.plot(epoch_avg_loss_train.keys(), epoch_avg_loss_train.values(), marker="o", linestyle="-", label="Training Loss", color="blue")
plt.plot(epoch_avg_loss_eval.keys(), epoch_avg_loss_eval.values(), marker="s", linestyle="--", label="Evaluation Loss", color="red")

# Labels and title
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training vs Evaluation Loss per Epoch (7000)")
plt.legend()
plt.grid()

# Show the plot
plt.show()


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
# train_csv_file = "./training_log/training_stats_3500_epoch10.csv"
# df_train = pd.read_csv(train_csv_file)

eval_csv_file = "./training_log/eval_stats_3500_epoch10.csv"
df_eval = pd.read_csv(eval_csv_file)

# Compute average loss per epoch
# epochs_train = df_train["epoch"]
# losses_train = df_train["loss"]

# Create a dictionary to store sum and count of losses for each epoch
# epoch_loss_train = {}
# for epoch, loss in zip(epochs_train, losses_train):
#     rounded_epoch = int(epoch)  # Convert fractional epoch to integer epoch number
#     if rounded_epoch not in epoch_loss_train:
#         epoch_loss_train[rounded_epoch] = {"sum": 0, "count": 0}
#     epoch_loss_train[rounded_epoch]["sum"] += loss
#     epoch_loss_train[rounded_epoch]["count"] += 1

# Compute average loss for each epoch
# epoch_avg_loss_train = {epoch: epoch_loss_train[epoch]["sum"] / epoch_loss_train[epoch]["count"] for epoch in epoch_loss_train}

# Compute average evaluation loss per epoch
epochs_eval = df_eval["epoch"]
losses_eval = df_eval["eval_loss"]

# Dictionary to store sum and count of evaluation losses per epoch
epoch_loss_eval = {}
for epoch, loss in zip(epochs_eval, losses_eval):
    rounded_epoch = int(epoch)  # Convert fractional epoch to integer epoch number
    if rounded_epoch not in epoch_loss_eval:
        epoch_loss_eval[rounded_epoch] = {"sum": 0, "count": 0}
    epoch_loss_eval[rounded_epoch]["sum"] += loss
    epoch_loss_eval[rounded_epoch]["count"] += 1

# Compute average evaluation loss per epoch
epoch_avg_loss_eval = {epoch: epoch_loss_eval[epoch]["sum"] / epoch_loss_eval[epoch]["count"] for epoch in epoch_loss_eval}

for epoch, avg_loss in epoch_avg_loss_eval.items():
    print(f"Epoch: {epoch}, Average Evaluation Loss: {avg_loss}")

    # Append the results to the CSV file
    results_df = pd.DataFrame({"epoch": [epoch], "eval_loss": [avg_loss]})
    results_df.to_csv("./training_log/eval_loss_ckpt.csv", mode='a', header=False, index=False)